#  <u> Concurrent Audit Automation </u>
On a daily basis, foreign exchange (FX) transactions data is monitored for compliance with RBI regulations pertaining to confirmation of FX deal. This includes a number of FX deal attributes (e.g. Transaction Reference, Counterparty Name, Deal Date, Value Date, Primary Currency, Counter Currency, Primary Currency Amount, Counter Currency Amount, etc.).
<br><br>
On an average, transaction testers have to manually review: 1250-1500 underlying documents with 12-15 attributes in each underlying document resulting in physical verification of 15000-22500 data points.
<br><br>
We have developed a transaction testing solution that can automate manual review by converting unstructured data in scanned “.pdf” files into structured data followed by comparison with the system generated deal listing. We have used state of the art machine learning algorithms, natural language processing techniques and self developed algorithms that can automate the process of verification of documents and simultaneously creating an assessment report that summarizes the faults and anomalies encountered while verification.

### Importing necessary libraries and the GOOGLE CREDENTIAL key for accessing CLOUD VISION API

In [53]:
import tkinter as tk
from tkinter import font
from tkinter import filedialog as fd

import sys 
import os 
import io
import re

import numpy as np
import pandas as pd

from PIL import Image, ImageOps
from pdf2image import convert_from_path
from difflib import SequenceMatcher

credential_path = 'E:\GCP\macbook-2c718e707810.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path
# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types

from toolbox import *
#from toolbox import detect_document, find_ref, amt, currency, givedates, changeformat

### Fields specified by user (Document and system dump path)

In [54]:
# Path of the document to be processed
pdf_path = '1681869232.pdf'

# Path to system dump
dmp_path = 'System dump.xlsx'

### Conversion of PDF to image
The PDF document is converted and saved in the form of image, which is further processed to enhance its quality and used to extract features from the document.

In [55]:
pdf_name = pdf_path
system_dump = dmp_path

img_name = 'img'

pages = convert_from_path(pdf_name, 500)
# Padding of image is done to make OCR more robust on texts printed on the edges
border = 150
image_counter = 1
for page in pages: 
    filename = img_name+str(image_counter)+".jpg"
    page = ImageOps.expand(page, border=border, fill = 'white')
    page.save(filename, 'JPEG') 
    image_counter = image_counter + 1 
filelimit = image_counter-1

###  Recognition of text and other important features of the document using Cloud Vision API

In [56]:
"""
Data is returned in two formats
1. data: Contains data in the form of blocks of text
2. data1: a sigle string containing all the text present in the document
"""

data =[]
for i in range(1, filelimit + 1): 
    filename = img_name+str(i)+".jpg"
    text = detect_document(filename)
    data.append(text)

strf = ''
for i in range(len(data)):
    for j in range(len(data[i])):
        for k in range(len((data[i])[j])):
            strf += ((((data[i])[j])[k]).strip(" "))
            strf += ' '
data1 = strf
data1 = data1.lower()

In [57]:
print(data)
print(data1)

[[['Ref', '1681869232'], ['Covering', 'Schedule', '-', 'Import'], ['XYZ', 'Bank'], ['Our', 'ref', '.', '5371LIF1800038'], ['Suprem', '.', '3rd', 'FL', ',', 'CTS', 'No', '.', '1337', '/', '2', ',', 'S', '.', 'No', '.', 'XY', 'House', ',', 'Hazarimal', 'Somani', 'Marg', ',', 'Fort', 'Mumbai', '-', '1', 'Banking', '&', 'Other', 'Financial', 'Services', 'Swift', ':', 'DEUTINBSPUN'], ['LC', 'no', '.', '537LCF1800035'], ['CUSTOMER', 'INSTRUCTION'], ['Tel', 'Fax'], ['Amount', 'USD', '456', ',', '000', '.', '00', 'Tenor', '86', 'DAYS', 'From', 'dale', 'of', 'Invoice', 'Corr', '.', 'ref', '.'], ['28', 'APR', '2018'], ['Applicant', '456', 'INDIA', 'PRIVATE', 'LIMITED'], ['Beneficiary', 'KUIBYSHEVAZOT', 'TRADING', '(', 'HK', ')', 'CO', '.', ',', 'LTD', ',', 'UNIT', '1109', ',', '11', '/', 'F', ',', 'KOWLOON', 'CENTRE', ',', '33', 'ASHLEY', 'ROAD', ',', 'TSIMSHATSUI', 'KOWLOON', ',', 'HONG', 'KONG', '.'], ['UNIT', '601', ',', '6TH', 'FLOOR', ',', 'CELLO', 'PLATINA', 'FERGUSSON', 'ROAD', ',', 'MODE

### Extraction of attributes from document.
Attributes are extracted using self developed algorithms stored in toolbox.py file.<br>
For detailed description and working of algorithms see toolbox.py file.

In [58]:
ref_no = find_ref(data1)
invoice_amt = amt(data1)
cur_list = currency(data1)
sub = organisation(data)

# Extraction of dates
dlist = datestring(data1)
invdate = dlist[0]
setdate = dlist[1]

In [59]:
# Verification of extracted fields
print(ref_no)
print(invoice_amt)
print(invdate)
print(setdate)

1681869232
456000.0
28-April-2018
07-July-2018


### Information from System dump is extracted from the excel file
Once the reference number is extracted from the file, this information is used to verify whether the information about the same document is present in the system dump. If the information is present then all the fields corresponding to the reference number are extracted and stored in the variables for further processing.

In [60]:
# Reading Excel file
sys_dump = pd.read_excel(system_dump, index_col=0)

# Index of the row is extracted which belongs to the extracted reference number
idx = sys_dump.index[sys_dump['REF_ID'] == ref_no].tolist()

sys_counter_party = (sys_dump.loc[idx[0]]['COUNTERPARTY_FULLNAME']).lower()
sys_trade_date = (sys_dump.loc[idx[0]]['TRADE_DATE'])
sys_settle_date = (sys_dump.loc[idx[0]]['SETTLEMENT_DATE'])
sys_cur1 = (sys_dump.loc[idx[0]]['CURRENCY1']).lower()
sys_cur2 = (sys_dump.loc[idx[0]]['CURRENCY2']).lower()
sys_amt = float(sys_dump.loc[idx[0]]['NOTIONAL1'])
sys_amt = abs(sys_amt)
# Stamp date also to be included

In [61]:
print(sys_counter_party)
print(sys_trade_date)
print(sys_settle_date)
print(sys_cur1)
print(sys_cur2)
print(sys_amt)

456 india private limited
15-JUN-2018
11-JUL-2018
usd
inr
456000.0


### Algorithm for the verification of attributes extracted from the document
Each attribute is verified based on the guidelines provided by the organisers. Each attribute has an associated flag variable which acts as an indication of the status of accuracy of the attribute. 

In [62]:
# Flag will be raised when there is exception
amt_f = 0
ivd_f = 0
pd_f = 0
cur_f = 0
org_f = 0
stm_f = 0

valid = 0

# Processing for date
td = changeformat(sys_trade_date)
sd = changeformat(sys_settle_date)
listx = givedates(data1)

# Verification for Amount
if(invoice_amt >= sys_amt):
    amt_f = 0
else:
    amt_f = 1



# Verification for currency
if((sys_cur1 in cur_list) or (sys_cur2 in cur_list)):
    cur_f = 0
else:
    cur_f = 1


# Verification for organisation
if(sys_counter_party in sub):
    org_f = 0
else:
    org_f = 1


# Trade(sys_dmp) - Invoice(doc) > 0
delta1 = date(td[0], td[1], td[2]) - date(listx[0][0], listx[0][1], listx[0][2])
if(delta1.days > 0):
    ivd_f = 0
else:
     ivd_f = 1



# Payment date(doc) - Settlement date > 0
delta2 = date(listx[1][0], listx[1][1], listx[1][2]) - date(sd[0], sd[1], sd[2])
if(delta2.days > 0):
    pd_f = 0
else:
    pd_f = 1


stamp_message = ''
#  Stamp(doc) - Trade <= 15
if(len(listx) >= 3):
    delta3 = date(listx[-1][0], listx[-1][1], listx[-1][2]) - date(td[0], td[1], td[2])
    if(delta3.days <= 15):
        stm_f = 0
    else:
        stamp_message = ' Stamp date did not match.'
        stm_f = 1
else:
    stamp_message = ' Stamp not found.'
    stm_f = 1



# Document verified or not
if(amt_f == 0 and ivd_f == 0 and pd_f == 0 and cur_f == 0 and org_f == 0 and stm_f == 0):
    valid = 0
else:
    valid = 1



if(valid==1):
    validation = 'Not Verified !!'

    message = ''
    if(amt_f == 1):
        message += ' Amount,'
    if(ivd_f == 1):
        message += ' Invoice date,' 
    if(pd_f == 1):
        message += ' Payment date,'
    if(cur_f == 1):
        message += ' Currency,'
        
    if(org_f == 1):
        message += ' Organisation,'
    message +=' is/are not verified in the document. /n'
    message += stamp_message

else:
    validation = 'Verified !!'
    message = 'All the fields are verified'

### <u> Saving the extracted attributes and assessment report in the form of DataFrame </u>
The extracted attributes which includes - <br>
a) Reference number<br>
b) Invoice Date<br>
c) Settlement date<br>
d) Counterparty Name<br>
e) Amount<br>
f) Information About the verification<br>
g) Detailed Assessment Report

In [63]:
#df1 = pd.read_excel("output.xlsx", index_col=None)
df2 = pd.DataFrame({"Reference no.":[ref_no], 
                    "Invoice Date":[invdate.upper()],
                    "Settlement Date":[setdate.upper()],
                    "Counterparty Name":[sys_counter_party.upper()], 
                    "Amount":[invoice_amt],
                    "Verification":[validation.upper()],
                    "Message":[message.upper()]})

In [65]:
df2

,Reference no.,Invoice Date,Settlement Date,Counterparty Name,Amount,Verification,Message
0,1681869232,28-APRIL-2018,07-JULY-2018,456 INDIA PRIVATE LIMITED,456000.0,NOT VERIFIED !!,"PAYMENT DATE, ORGANISATION, IS/ARE NOT VERIFI..."
